# Serving Runtime Remote Artifacts

In [1]:
import mlrun
import os

# Initialize the MLRun project object
project = mlrun.get_or_create_project('remote-artifacts',user_project=True,context='./')

# Required credentials :
# AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY, GOOGLE_APPLICATION_CREDENTIALS, S3_BUCKET
AWS_ACCESS_KEY_ID = os.environ.get('AWS_ACCESS_KEY_ID', None)
AWS_SECRET_ACCESS_KEY = os.environ.get('AWS_SECRET_ACCESS_KEY', None)
GOOGLE_APPLICATION_CREDENTIALS = os.environ.get('GOOGLE_APPLICATION_CREDENTIALS', None)
assert AWS_ACCESS_KEY_ID != None and AWS_SECRET_ACCESS_KEY != None and GOOGLE_APPLICATION_CREDENTIALS != None, "please provide credentials"

secrets = {'AWS_ACCESS_KEY_ID': AWS_ACCESS_KEY_ID,
           'AWS_SECRET_ACCESS_KEY':AWS_SECRET_ACCESS_KEY,
           'GOOGLE_APPLICATION_CREDENTIALS': GOOGLE_APPLICATION_CREDENTIALS}

project.set_secrets(secrets=secrets, provider='kubernetes')

S3_BUCKET = os.environ.get('S3_BUCKET', 'testbucket-igz')

project.artifact_path = os.path.join('s3://', S3_BUCKET + '/remote-artifacts/')

> 2023-01-11 08:48:30,346 [info] loaded project remote-artifacts from MLRun DB


In [2]:
# Setting dummy serving function
project.set_function(mlrun.new_function(name='serving', kind='serving',image='mlrun/mlrun', command = []))
project.get_function('serving').spec.build.commands = ['pip install xgboost==1.6.2']
project.get_function('serving').set_envs(secrets)
# adding our remote source model
model_artifact = project.get_artifact('log-transactions-log_transactions_clf_model')
model_path = model_artifact.get_target_path() + model_artifact.model_file

project.get_function('serving').add_model(key = 'my_model',
                                          class_name = "mlrun.frameworks.xgboost.XGBoostModelServer",
                                          model_path = model_path)

# mounting
project.get_function('serving').apply(mlrun.auto_mount())
project.get_function('serving').apply(mlrun.platforms.mount_s3())

> 2023-01-11 08:48:30,443 [warning] it is recommended to use k8s secret (specify secret_name), specifying the aws_access_key/aws_secret_key directly is unsafe


In [3]:
server = project.get_function('serving').to_mock_server()

> 2023-01-11 08:48:30,460 [warning] run command, file or code were not specified
> 2023-01-11 08:48:41,358 [info] model my_model was loaded
> 2023-01-11 08:48:41,359 [info] Loaded ['my_model']


In [4]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

X,y = load_iris(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

server.test(body={'inputs': X_test.tolist()[:3]})

{'id': '43c218b09bed42ad8a6372c078e76100',
 'model_name': 'my_model',
 'outputs': [2, 1, 0]}

In [5]:
project.deploy_function('serving')

> 2023-01-11 08:48:41,418 [info] Starting remote function deploy
2023-01-11 08:48:41  (info) Deploying function
2023-01-11 08:48:41  (info) Building
2023-01-11 08:48:41  (info) Staging files and preparing base images
2023-01-11 08:48:41  (info) Building processor image
2023-01-11 08:49:57  (info) Build complete
2023-01-11 08:50:51  (info) Function deploy complete
> 2023-01-11 08:50:52,679 [info] successfully deployed function: {'internal_invocation_urls': ['nuclio-remote-artifacts-admin-serving.default-tenant.svc.cluster.local:8080'], 'external_invocation_urls': ['remote-artifacts-admin-serving-remote-artifacts-admin.default-tenant.app.vmdev94.lab.iguazeng.com/']}


DeployStatus(state=ready, outputs={'endpoint': 'http://remote-artifacts-admin-serving-remote-artifacts-admin.default-tenant.app.vmdev94.lab.iguazeng.com/', 'name': 'remote-artifacts-admin-serving'})

In [6]:
import time
time.sleep(5)
project.get_function('serving').invoke('/v2/models/my_model/infer', body={'inputs': X_test.tolist()[:3]})

> 2023-01-11 08:50:57,779 [info] invoking function: {'method': 'POST', 'path': 'http://nuclio-remote-artifacts-admin-serving.default-tenant.svc.cluster.local:8080/v2/models/my_model/infer'}


{'id': '17fb8bc1-d7d6-483a-8bbc-2ac51c9bfceb',
 'model_name': 'my_model',
 'outputs': [2, 1, 0]}

# Pipeline remote access

In [7]:
%%writefile workflow.py
import mlrun
from kfp import dsl
import os

@dsl.pipeline(name="Mask Detection Pipeline")

def kfpipeline():
    # Get our project object:
    project = mlrun.get_current_project()
    
    project.get_function('log_transactions').apply(mlrun.platforms.set_env_variables({'GOOGLE_APPLICATION_CREDENTIALS':
                                                                                      os.environ.get('GOOGLE_APPLICATION_CREDENTIALS')}))
    project.get_function('log_transactions').apply(mlrun.platforms.mount_s3())
    project.get_function('log_transactions').apply(mlrun.auto_mount())
    run = project.run_function('log_transactions')
    
    model_artifact = project.get_artifact('log-transactions-log_transactions_clf_model')
    model_path = model_artifact.get_target_path() + model_artifact.model_file
    
    project.get_function('serving').add_model(key = 'my_model',
                                          class_name = "mlrun.frameworks.xgboost.XGBoostModelServer",
                                          model_path = model_path)
    
    project.deploy_function('serving').after(run)
    return

Overwriting workflow.py


In [8]:
# Register the workflow file:
workflow_name = "iris_workflow"
project.set_workflow(workflow_name, "workflow.py")

# Save the project:
project.save()

In [9]:
# Running with S3
project.run(name=workflow_name,watch=True, local=True, artifact_path=project.artifact_path)
project.save()

> 2023-01-11 08:51:19,773 [warning] it is recommended to use k8s secret (specify secret_name), specifying the aws_access_key/aws_secret_key directly is unsafe
> 2023-01-11 08:51:19,780 [info] starting run log-transactions-log_transactions uid=94fcc9a4974349cf9deedcf46c42e56c DB=http://mlrun-api:8080
> 2023-01-11 08:51:24,003 [info] encoded dataframe shape : (150, 4)
> 2023-01-11 08:51:25,046 [info] logged model : <class 'xgboost.sklearn.XGBClassifier'>
> 2023-01-11 08:51:33,650 [info] logged dataset shape (150, 5)


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
remote-artifacts-admin,...6c42e56c,0,Jan 11 08:51:19,completed,log-transactions-log_transactions,workflow=276510b776524fc7bda371d099b88b22v3io_user=adminkind=owner=adminhost=jupyter-6479d4cd8c-rnxrj,,,,encoded_iris-s3clf_modelmy_projectiris_dataset-s3


> 2023-01-11 08:51:33,784 [info] run executed, status=completed
> 2023-01-11 08:51:33,817 [info] Starting remote function deploy
2023-01-11 08:51:35  (info) Deploying function
2023-01-11 08:51:35  (info) Building
2023-01-11 08:51:40  (info) Staging files and preparing base images
2023-01-11 08:51:40  (info) Building processor image
2023-01-11 08:53:00  (info) Build complete
2023-01-11 08:53:14  (info) Function deploy complete
> 2023-01-11 08:53:14,946 [info] successfully deployed function: {'internal_invocation_urls': ['nuclio-remote-artifacts-admin-serving.default-tenant.svc.cluster.local:8080'], 'external_invocation_urls': ['remote-artifacts-admin-serving-remote-artifacts-admin.default-tenant.app.vmdev94.lab.iguazeng.com/']}


uid,start,state,name,parameters,results
...6c42e56c,Jan 11 08:51:19,completed,log-transactions-log_transactions,,


> 2023-01-11 08:53:15,020 [info] started run workflow remote-artifacts-admin-iris_workflow with run id = '276510b776524fc7bda371d099b88b22' by local engine


In [10]:
# Running with GCS
project.run(name=workflow_name,watch=True, local=True, artifact_path='gs' + project.artifact_path[2:])

> 2023-01-11 08:53:15,272 [warning] it is recommended to use k8s secret (specify secret_name), specifying the aws_access_key/aws_secret_key directly is unsafe
> 2023-01-11 08:53:15,278 [info] starting run log-transactions-log_transactions uid=ee85d3f6e0da4411869e62b8a45696a3 DB=http://mlrun-api:8080
> 2023-01-11 08:53:18,018 [info] encoded dataframe shape : (150, 4)
> 2023-01-11 08:53:21,579 [info] logged model : <class 'xgboost.sklearn.XGBClassifier'>
> 2023-01-11 08:53:24,021 [info] logged dataset shape (150, 5)


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
remote-artifacts-admin,...a45696a3,0,Jan 11 08:53:15,completed,log-transactions-log_transactions,workflow=9a04e1a2584748b48e2e45022104884ev3io_user=adminkind=owner=adminhost=jupyter-6479d4cd8c-rnxrj,,,,encoded_iris-gsclf_modelmy_projectiris_dataset-gs


> 2023-01-11 08:53:24,162 [info] run executed, status=completed
> 2023-01-11 08:53:24,197 [info] Starting remote function deploy
2023-01-11 08:53:24  (info) Deploying function
2023-01-11 08:53:24  (info) Building
2023-01-11 08:53:24  (info) Staging files and preparing base images
2023-01-11 08:53:24  (info) Building processor image
2023-01-11 08:54:39  (info) Build complete
2023-01-11 08:54:53  (info) Function deploy complete
> 2023-01-11 08:54:53,708 [info] successfully deployed function: {'internal_invocation_urls': ['nuclio-remote-artifacts-admin-serving.default-tenant.svc.cluster.local:8080'], 'external_invocation_urls': ['remote-artifacts-admin-serving-remote-artifacts-admin.default-tenant.app.vmdev94.lab.iguazeng.com/']}


uid,start,state,name,parameters,results
...a45696a3,Jan 11 08:53:15,completed,log-transactions-log_transactions,,


> 2023-01-11 08:54:53,798 [info] started run workflow remote-artifacts-admin-iris_workflow with run id = '9a04e1a2584748b48e2e45022104884e' by local engine


9a04e1a2584748b48e2e45022104884e